In [1]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = 'api_key'

In [2]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/medical-o1-verifiable-problem")
print(dataset)


c:\medical\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetDict({
    train: Dataset({
        features: ['Open-ended Verifiable Question', 'Ground-True Answer'],
        num_rows: 40644
    })
})


In [3]:
from langchain.document_loaders import HuggingFaceDatasetLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
import os

dataset = "FreedomIntelligence/medical-o1-verifiable-problem"
loader = HuggingFaceDatasetLoader(dataset,page_content_column="Open-ended Verifiable Question")
data = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
documents = text_splitter.split_documents(data)

In [4]:
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_14980\95610984.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")


In [5]:
vectorstore = FAISS.from_documents(documents,embeddings)

In [11]:
query = 'Which of the following ligaments is not classified as an Extrinsic Laryngeal Membrane/Ligament: Hyoepiglottic, Cricotracheal, Thyrohyoid, or Cricothyroid?'

retriever = vectorstore.as_retriever(search_kwargs={'k': 1}) 
docs = retriever.get_relevant_documents(query)

template = '''You are a helpful medical assistant. Answer the question using the given context.

Context:
{context}

Question: {question}
"
'''

prompt = ChatPromptTemplate.from_template(template)

repo_id = "deepseek-ai/DeepSeek-R1"
llm = HuggingFaceHub(
    repo_id=repo_id, 
    model_kwargs={"temperature": 0.2, "max_length": 256}  
)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

chain = prompt | llm | StrOutputParser()

# Run
response = chain.invoke({'context': format_docs(docs), 'question': query})
response


c:\medical\venv\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


'Human: You are a helpful medical assistant. Answer the question using the given context.\n\nContext:\n"Which of the following ligaments is not classified as an Extrinsic Laryngeal Membrane/Ligament: Hyoepiglottic, Cricotracheal, Thyrohyoid, or Cricothyroid?"\n\nQuestion: Which of the following ligaments is not classified as an Extrinsic Laryngeal Membrane/Ligament: Hyoepiglottic, Cricotracheal, Thyrohyoid, or Cricothyroid?\n"\nAnswer: Cricotracheal'